In [1]:
# let's try to run LSTM on RSRP values from ns-3 dual strip simulation

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split


In [3]:
# First, read and prepare RSRP data
file = 'DlRsrpSinrStats_hom-0_ttt-64.txt'
h = open(file, 'r')
hlines = h.readlines()

base = []
for line in hlines:
    base.append(line.split())

# Organize data frame
base = pd.DataFrame(base)

base.shape

(195970, 8)

In [4]:
# transform RSRP from linear to dB

base.drop(columns=[1, 3, 6, 7], inplace=True)

base.columns=['time', 'IMSI', 'rsrp', 'sinr']

base = base.iloc[1:]

base['rsrp'] = np.log10(base['rsrp'].values.astype(float))*10

base.shape

(195969, 4)

In [5]:
# get only RSRP values from 1 UE as time series
myrsrp = []
myrsrp = base.loc[base['IMSI'].astype(int)==12, 'rsrp']
myrsrp.reset_index(drop=True, inplace=True)
myrsrp = pd.DataFrame(myrsrp).values

myrsrp.shape

(9800, 1)

In [7]:
# train and test split 

rsrptrain, rsrptest = train_test_split(
    myrsrp, test_size=0.7, random_state=42
    )

In [8]:
# Training phase
# initialing variables

prev = []
real_rsrp = []

# filling for 100-sample prediction
for i in range(100, rsrptrain.size):
    prev.append(rsrptrain[i-100:i, 0])
    real_rsrp.append(rsrptrain[i, 0])

# adapting formats (only 1 dimension)
prev, real_rsrp = np.array(prev), np.array(real_rsrp)

print('prev.shape',prev.shape)
print('real_rsrp.shape',real_rsrp.shape)
print("reshape..")

prev = np.reshape(prev, (prev.shape[0], prev.shape[1], 1))
print('prev.shape',prev.shape)


prev.shape (2840, 100)
real_rsrp.shape (2840,)
reshape..
prev.shape (2840, 100, 1)


In [9]:
# starting regressor
regressor = Sequential()
regressor.add(LSTM(units = 120, return_sequences = True, input_shape = (prev.shape[1], 1)))

# using dropout to avoid overfitting
regressor.add(Dropout(0.3))

# more layers
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))

# more layers
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))

# more layers
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.3))

# final layer
regressor.add(Dense(units = 1, activation = 'linear'))

regressor.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 120)          58560     
_________________________________________________________________
dropout (Dropout)            (None, 100, 120)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 50)           34200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 50)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 50)           20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 50)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)               

In [ ]:

# compiling
regressor.compile(optimizer = 'rmsprop', loss = 'mean_squared_error', metrics = ['mean_absolute_error'])
regressor.fit(prev, real_rsrp, epochs = 25, batch_size = 32)

# testing phase
# preparing inputs for test
inputs = myrsrp_norm[len(myrsrp_norm) - len(rsrptest) - 100:]
inputs = inputs.reshape(-1, 1)

# loop for filling variable
x_test = []
for i in range (100, inputs.size):
    x_test.append(inputs[i-100:i, 0])

# format adapting
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

prediction = regressor.predict(x_test)

# undo normalization for better viewing our results
prediction = scaler.inverse_transform(prediction)

# get real RSRP test values to plot and compare
real_rsrp_test = myrsrp[8821:9800, :]
mae = mean_absolute_error(real_rsrp_test, prediction)

# visualization
plt.plot(real_rsrp_test, color = 'red', label = 'Real RSRP')
plt.plot(prediction, color = 'blue', label = 'Prediction')
plt.title('RSRP values prediction')
plt.xlabel('Time (samples)')
plt.ylabel('RSRP (dB)')
plt.legend()
plt.grid()
plt.show()